In [274]:
import pennylane as qml
import numpy as np

from qiskit.quantum_info import Operator

from src.CPQAOA import CP_QAOA
from src.QAOA import QAOA
from src.Chain import Chain
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       get_qubo, 
                       normalized_cost, 
                       qubo_limits, 
                       check_qubo)
from src.Tools import get_qiskit_H

In [275]:
n_wires = 17
init_strat = np.array([1 if i%2 == 1 else 0 for i in range(n_wires)])
k = n_wires // 2
my_indices = [(i, i+1) for i in range(n_wires-1)]
n_layers = 4
alpha = 0.5
seed = 0
gamma_vals, beta_vals = np.random.uniform(-2*np.pi, 2*np.pi, n_layers), np.random.uniform(-2*np.pi, 2*np.pi, n_layers)
#TODO: Check out the 'lightning.gpu' plugin, which is a fast state-vector simulator offloading to the NVIDIA cuQuantum SDK for GPU accelerated circuit simulation. (not supported on windows...)
dev = qml.device('lightning.qubit', wires=n_wires)

In [276]:
 # Defining topology
my_chain = Chain(N_qubits=n_wires)
my_chain.set_initialization_strategy(strategy=init_strat)

# Deciding between grid and 1d chain topology
my_topology = my_chain

# Generating random problem instance 
expected_returns, covariances = portfolio_metrics(n=n_wires, seed=seed)

# Retrieving C_min, C_max and corresponding states for original portfolio problem
constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=n_wires,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
#full_space_max_cost = full_result['c_max']
portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])

# Generating QUBO corresponding to current problem instance
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances, 
                     alpha=alpha,
                     lmbda=lmbda+1e-8, # Adding small constant purposely
                     k=k)
QUBO_limits = qubo_limits(Q=Q,offset=offset)
qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])


if not portfolio_subspace_min_state_str == qubo_min_state_str:
    raise RuntimeError(f'portfolio_subspace_min_state_str: {portfolio_subspace_min_state_str}, qubo_min_state_str={qubo_min_state_str}'+f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not np.isclose(qubo_min_cost,portfolio_subspace_min_cost):
    raise RuntimeError(f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not qubo_max_cost >= portfolio_subspace_max_cost:
    raise RuntimeError(f'Max. cost of qubo: {qubo_max_cost}, max. cost of portfolio subspace: {portfolio_subspace_max_cost} (should be qubo max. >= constrained portfolio max)')

In [277]:
# unitary operator U_B with parameter beta
def U_B(beta, n_wires):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)

def RZ(angle, qubit):
    qml.RZ(phi=angle, wires=qubit)

def RZZ(angle, qubit_1, qubit_2):
    qml.CNOT(wires=[qubit_1, qubit_2])
    qml.RZ(phi=angle, wires=qubit_2)
    qml.CNOT(wires=[qubit_1, qubit_2])
# unitary operator U_C with parameter gamma
def U_C(gamma, indices):
    for pair in indices:
        qubit_1, qubit_2 = pair[0], pair[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])
        
@qml.qnode(dev)
def circuit(gammas, betas, indices, n_layers, n_wires):
    # apply Hadamards to get the n qubit |+> state
    for wire in range(n_wires):
        qml.Hadamard(wires=wire)
        
    # p instances of unitary operators
    for i in range(n_layers):
        U_C(gammas[i], indices)
        U_B(betas[i], n_wires)

    return qml.state()

def string_to_array(string_rep: str) -> np.ndarray:
    return np.array([int(bit) for bit in string_rep]).astype(np.float64)

def qubo_cost(state: np.ndarray, QUBO_matrix: np.ndarray) -> float:
    return np.dot(state, np.dot(QUBO_matrix, state))

def int_to_fixed_length_binary_array(number, num_bits):
    # Convert the number to binary and remove the '0b' prefix
    binary_str = bin(number)[2:]
    # Pad the binary string with zeros if necessary
    return binary_str.zfill(num_bits)

def get_counts(state_vector: np.ndarray) -> dict:
    n_qubits = int(np.log2(len(state_vector)))
    return {int_to_fixed_length_binary_array(number=idx, num_bits=n_qubits): np.abs(state_vector[idx])**2 for idx in range(len(state_vector))}

def cost(Q, state_vector: np.ndarray) -> float:
    counts = get_counts(state_vector=state_vector)
    return np.mean([probability * qubo_cost(state=string_to_array(bitstring), QUBO_matrix=Q) for
                        bitstring, probability in counts.items()]) 


In [278]:
state = circuit(gamma_vals, beta_vals, my_indices, n_layers, n_wires)

In [279]:
c = cost(Q=Q, state_vector=state)
c

-0.043870270390343184

In [ ]:
from typing import *

class Pennylane_QAOA:
    def __init__(self,
                 N_qubits,
                 layers,
                 QUBO_matrix,
                 QUBO_offset,
                 constraining_mixer: bool = False,
                 Topology: Union[Grid, Chain] = None,
                 normalize_cost: bool = False):
        self.n_qubits = N_qubits
        self.layers = layers
        self.QUBO_matrix = QUBO_matrix
        self.J_list, self.h_list = get_ising(Q=QUBO_matrix, offset=QUBO_offset)
        self.constraining_mixer = constraining_mixer
        if constraining_mixer:
            if Topology is None:
                raise ValueError(f'"Topology" should be provided when "constraining_mixer" is True...')
            self.mixer_qubit_indices = Topology.get_NN_indices()
            self.initialization_strategy = Topology.get_initialization_strategy()
        self.normalize_cost = normalize_cost

        self.counts = None